In [38]:
import os
from my_tools import *
from dependency import *
from instrumentation import *
from insert_patch import *
from program_synthesis import *
from get_heap_object import *
import argparse
import json
import time

In [ ]:
parser = argparse.ArgumentParser(description='file name')
parser.add_argument('f',type=str, help='file name')
args = parser.parse_args()
filename=args.f

In [21]:
# input the name of buggy program
def main(filename):
    t0=time.time()
    # delete comment
    os.system(f"gcc -fpreprocessed -dD -E {filename} | sed \"1d\" > _{filename}")
    # fuzzing
    print("fuzzing")
    os.system(f"clang-12 -g -fsanitize=address,fuzzer _{filename}")
    err_num=0
    for _ in range(10):
        os.system(r"./a.out -max_total_time=5 -max_len=2 2>fuzzer_result")
        err_path=get_fuzzer_result("fuzzer_result")
        if len(err_path)>err_num:
            err_num=len(err_path)
            os.system("cat fuzzer_result > tmp")
    os.system("cat tmp > fuzzer_result")
    os.system("rm -f tmp")
    print("fuzzing done")
    # delete include (pycparser can't analyze)
    os.system(f"cat _{filename} | sed \"/#/c\ \" > dep_{filename} ")
    
    # get dependency variable and the error
    dep,error_feature = get_dependency("fuzzer_result")
    _error_feature=error_feature.copy()
    with open('dependency.json', 'w') as f:
        json.dump(dep, f, indent=4)
    print("dependency done")
    # get file list (line)
    filelist = read_file(filename)
    # source instrumentation 
    inst_filename = instrument(dep)
    
    # initialization of instrumentation result
    syn_inf = get_synthesis_inf(dep)
    os.system(f"clang-12 -g -fsanitize=address,fuzzer {inst_filename}")
    # run 10 times to get more dynamic value
    for i in range(10):
        os.system(r"./a.out -max_total_time=5 -max_len=2 2>inst_result")
        add_dynamic_value(syn_inf,"inst_result",_error_feature,filelist)
    with open('syn_inf.json', 'w') as f:
        json.dump(syn_inf, f, indent=4) 
        
    print("collect dynamic value done")  
    print(f"current time: {time.time()-t0}")    
    print(f"have {len(error_feature)} error(s)")
    
    queue=[i for i in range(len(error_feature))]
    to_fix_list=[]
    # try to fix each error
    # for i in range(len(error_feature)):
    flag=0
    while queue and flag!=len(queue):
        i=queue.pop(0)
        err_dep = dep[i]
        err_fea = _error_feature[i]
        t1=time.time()
        t=0
        not_fixed=1 # flag
        while t<60 and not_fixed: # one error 1 min
            err_inf = syn_inf[i]
            # for each error, generate the patch of each function in the error patch 
            patch_cand = synthesis().synthesis(err_inf)
            # insert temporary variable in each function and keep it filelist
            patch = insert_tmp_var(filelist,patch_cand)
            # try to fix this error
            cur_filelist,not_fixed=fix(patch,err_dep,err_fea,error_feature,i,dep)
            # if failed, instrument again and get new dynamic variable
            if not_fixed:
                print("collect error test cases")
                add_dynamic_value(syn_inf,"cur_fuzzer_result",_error_feature,filelist)
                '''
                os.system(f"clang-12 -g -fsanitize=address,fuzzer {inst_filename}")
                for _ in range(10):
                    os.system(r"./a.out -max_total_time=5 -max_len=2 2>inst_result")
                    add_dynamic_value(syn_inf,"inst_result",_error_feature)
                '''
            t=time.time()-t1
        # if this error is fixed
        if not not_fixed:
            print(f"fixed error {i}, error feature: {err_fea}")
            # keep current patch
            filelist=cur_filelist.copy()
            write_file("cur_"+filename,cur_filelist)
            error_feature.remove(_error_feature[i])
            flag=0
            if queue:
                # clean dynamic value and collect new values
                clean_inf(syn_inf)
                new_instrument(cur_filelist.copy(),dep,inst_filename)
                print("collect new test cases")
                os.system(f"clang-12 -g -fsanitize=address,fuzzer {inst_filename}")
                for _ in range(10):
                    os.system(r"./a.out -max_total_time=5 -max_len=2 2>inst_result")
                    add_dynamic_value(syn_inf,"inst_result",_error_feature,filelist)
        else:
            # if time out, print error number
            print(f"failed to fix error {i}")
            queue.append(i)
            flag+=1

                
        os.system(r"rm -f leak*")
        os.system(r"rm -f crash*")
    # write file at last
    os.system(f"rm -f *_{filename}")
    os.system(f"rm -f *_result")
    os.system(f"rm a.out")
    os.system(f"rm -f check_and_instrument.c")
    write_file("result_"+filename,filelist)
    print(f"total time: {time.time()-t0}")

In [44]:
def fix(patch,err_dep,err_fea,error_feature,i,dep):
    """ 
    k: funcname
    v: filelist, patch, return location
    """
    # try the patch in each function
    for k,v in patch.items():
        if not v:
            continue
        cur_filelist=v["filelist"].copy()
        # get object type and name
        o,otype,line=get_error_object(err_dep,k)
        # if failed to get object information, try the patch in next function 
        if o == 0:
            continue
        # insert temporary variable to keap the object
        cur_filelist,o=insert_heap_object(cur_filelist.copy(),o,otype,line,i)
        # synthesis the patch: if(c)free(o);
        free=cur_patch(v["patch"],o)
        print(f"current patch is {free} in function {k}")
        # insert the patch before each return, to find the correct location
        for ret in v["ret"]:
            print(f"insert at {ret}")
            patch_line=int(ret["coord"].split(":")[1])
            # insert patch
            cand_filelist=insert_patch(cur_filelist.copy(),patch_line,free)
            # test patched program
            new_instrument(cand_filelist.copy(),dep,"check_and_instrument.c")
            #write_file("patched_"+filename,cand_filelist)
            os.system(f"clang-12 -g -fsanitize=address,fuzzer check_and_instrument.c")
            err_num=0
            f=0
            for _ in range(20):
                os.system(r"./a.out -max_total_time=5 -max_len=2 2>cur_fuzzer_result")
                err_path=get_fuzzer_result("cur_fuzzer_result")
                if len(err_path)>err_num:
                    f=1
                    err_num=len(err_path)
                    os.system("cat cur_fuzzer_result > tmp")
            if f:
                os.system("cat tmp > cur_fuzzer_result")
                os.system("rm -f tmp")
            # collect current error
            cur_err=get_error_feature(get_fuzzer_result("cur_fuzzer_result"))
            # same with the original error
            if error_feature == cur_err:
                print("repair a part of error")
                # repair a part of error, keep the current patch
                cur_filelist=cand_filelist.copy()
                continue
            # new error happened
            flag=0
            for err in cur_err:
                if err=="DF" or err=="UAF":
                    print("wrong patch/location")
                    flag=1
                    # wrong patch
                    break
                if err not in error_feature:
                    for e in error_feature:
                        if e[-1]==err[-1]:
                            print("wrong patch/location")
                            flag=1
                            # wrong patch
                            break
            if flag:
                continue
            # not in current error
            if err_fea not in cur_err:
                print("correct patch")
                return cand_filelist,0
    return None,1

In [ ]:
file=open("inst_result",'r')
inf=file.read()
for i in inf.split("\n"):
    if "instrument:" in i :
        print(i)

In [ ]:
{'func': {'patch': 'tmp_a==5',
   'ret': [{'coord': 'dep__src.c:12:9'}, {'coord': 'dep__src.c:12:16'}]}}

In [4]:
syn_inf=get_synthesis_inf(dep,"inst_result")

In [48]:
syn_inf

[{'error': [],
  'var': {'func': [{'state': 'input',
     'type': 'int',
     'name': 'a',
     'coord': 'dep__src.c:5:14',
     'value': [],
     'ret': [{'coord': 'dep__src.c:10:9'},
      {'coord': 'dep__src.c:12:9'},
      {'coord': 'dep__src.c:12:16'}]}]}}]

In [ ]:
add_dynamic_value(syn_inf,"inst_result")

In [45]:
def clean_inf(syn_inf,i):
    syn_inf[i]["error"]=[]
    for v in syn_inf[i]["var"].values():
        for var in v:
            var["value"]=[]
    

In [47]:
clean_inf(syn_inf,0)

In [ ]:
syn_inf

In [ ]:
get_dynamic_value("inst_result")

In [6]:
def get_error_object(dep,func):
    if dep[func]["object"][1]==dep["object"][1]:
        return dep[func]["object"]
    else:
        o = dep["object"][0]
        otype = dep["object"][1]
        line = dep[func]["object"][2]
        if "->" in o:
            namelist=o.split("->")
            if "." in namelist[0]:
                name=dep[func]["object"][0]+"."+".".join(namelist[0].split(".")[1:])+"->"+"->".join(namelist[1:])
            else:
                name=dep[func]["object"][0]+"->"+"->".join(namelist[1:])
        elif "." in o:
            namelist=o.split(".")
            name=dep[func]["object"][0]+"."+".".join(namelist[1:])
        else:
            return 0,0,0
        return (name,otype,line)
    

In [24]:
dep

[{'func': {'dep': [{'name': 'a',
     'state': 'input',
     'type': 'int',
     'coord': 'dep__src.c:5:14'},
    {'name': 'b', 'state': 'var', 'type': 'int*', 'coord': 'dep__src.c:7:13'}],
   'ret': [{'coord': 'dep__src.c:10:9'},
    {'coord': 'dep__src.c:12:9'},
    {'coord': 'dep__src.c:12:16'}],
   'object': ('p', 'int*', 6)},
  'error_object': ('p', 'int*', 6)}]

In [ ]:
data={
        "error":[1,1,1,1,1,1,1,1,1,1],
        "var":{"func":[
            {
                "state":"input",
                "type":"int",
                "name":"a",
                "coord":2,
                "value":[2,2,6,6,6,5,4,3,4,10],
                "ret":[]
            },
            {
                "state":"input",
                "type":"int",
                "name":"b",
                "coord":2,
                "value":[0,1,-2,-1,0,2,4,6,6,1],
                "ret":[]
            },
            {
                "state":"var",
                "type":"int",
                "name":"c",
                "coord":6,
                "value":[2,3,4,5,6,7,8,9,10,11],
                "ret":[]
            }
        ]}
    
}

In [32]:
a="n"
len(a)

1

In [35]:
f=open("instrumentation.json","r")
a=json.load(f)

In [52]:
a=[None,None]
a

[None, None]

In [53]:
s=["a","b"]
"".join(s)

'ab'

In [55]:
"1".isdigit()

True